In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [60]:
from datasets import load_dataset
import pandas as pd
import re
from transformers import pipeline

# Step 1: Load Dataset
ds = load_dataset("MohammadOthman/mo-customer-support-tweets-945k")
data = ds['train'].select(range(10000)).to_pandas()


In [61]:
data

,output,input
0,I would love the chance to review the account ...,is the worst customer service
1,H there! we would definitely like to work with...,y’all lie about your “great” connection. 5 bar...
2,We understand your concerns and we would like ...,Since I signed up with you....Since day 1
3,This is saddening to hear. Please shoot us a D...,You gonna magically change your connectivity f...
4,Please send me a private message so that I can...,"whenever I contact customer support, they tell..."
...,...,...
9995,Totally worth it.,braving 40 degree weather in this for a 3 doll...
9996,Woo! Hope you enjoyed it.,Guac was not extra today thanks to Boorito
9997,that is concerning. Can you please share more ...,Went to last night.....well had to take it bac...
9998,Talk about having some of the best views in th...,Pretty sunrise over cottonball clouds. Thanks ...


In [63]:
from transformers import pipeline

# Load sentiment classifier
sentiment_classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Function to get sentiment
def get_sentiment(text):
    result = sentiment_classifier(text)[0]
    return result['label'], result['score']

# Apply sentiment analysis
data['sentiment_label'], data['sentiment_score'] = zip(*data['output'].apply(get_sentiment))

# Filter positive responses (LABEL_2 is positive)
positive_data = data[data['sentiment_label'] == 'LABEL_2']
print(f"Number of positive responses: {len(positive_data)}")

Device set to use mps:0


Number of positive responses: 3396


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import torch

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define dataset class
class PositiveResponseDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = "respond to query: " + self.data.iloc[idx]['input']
        target_text = self.data.iloc[idx]['output']
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors="pt", max_length=512, truncation=True)
        return {"input_ids": input_ids.squeeze(), "labels": target_ids.squeeze()}

# Create dataset and dataloader
dataset = PositiveResponseDataset(positive_data)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Set up device and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Fine-tune the model
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

sentiment sentence genrator with fine tunning of model ends here



In [25]:
# # Step 4: Sentiment Classification (Weak Labels)
# classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# sample_texts = data['clean_conversation'][:].tolist()
# predictions = classifier(sample_texts, truncation=True)

# # Step 5: Map to Satisfaction Labels
# def map_sentiment_to_satisfaction(label):
#     if label == "POSITIVE":
#         return "highly_satisfied"
#     else:
#         return "unhappy"

# data.loc[:, 'satisfaction'] = [map_sentiment_to_satisfaction(pred['label']) for pred in predictions]

# Step 6: Show Labeled Sample
# print(data[['input', 'output', 'satisfaction']].head(20))

# data.to_csv('processed_data.csv', index= False)


In [27]:
from datasets import load_dataset
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModel
import torch

# Load Dataset
ds = load_dataset("MohammadOthman/mo-customer-support-tweets-945k")
data = ds['train'].select(range(100000)).to_pandas()

# Preprocess text: lowercase, remove special characters
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Apply preprocessing
data['input'] = data['input'].apply(preprocess_text)
data['output'] = data['output'].apply(preprocess_text)
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

def get_embeddings(text):
    # Check if the input text is empty or whitespace-only
    if not text or not text.strip():
        print("Warning: Empty text input provided.")
        return torch.zeros(1, 768)  # Return a zero vector with the expected embedding size
    
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    # Check if tokenization produced any tokens
    if inputs['input_ids'].shape[1] == 0:
        print("Warning: No tokens generated for input:", text)
        return torch.zeros(1, 768)  # Return a zero vector
    
    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Verify the output tensor
    if outputs.last_hidden_state.numel() == 0:
        print("Error: Model output is empty for input:", text)
        return torch.zeros(1, 768)
    
    # Compute embeddings (mean of last hidden state)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Shape: [1, 768]
    return embeddings

# Test it
text = ""  # Empty input
emb = get_embeddings(text)  # Should return a zero tensor without error
print(emb.shape)  # Expected: torch.Size([1, 768])

text = "Hello, world!"
emb = get_embeddings(text)  # Should work normally
print(emb.shape)  # Expected: torch.Size([1, 768])

torch.Size([1, 768])


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CVAE(nn.Module):
    def __init__(self, input_dim=768, latent_dim=128, condition_dim=768):
        super(CVAE, self).__init__()
        # Encoder: input + condition -> latent space
        self.encoder = nn.Sequential(
            nn.Linear(input_dim + condition_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim * 2)  # Outputs mu and logvar
        )
        # Decoder: latent + condition -> reconstructed output
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
            nn.Sigmoid()
        )
        self.latent_dim = latent_dim

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, condition):
        # Encode
        enc_input = torch.cat([x, condition], dim=1)
        enc_output = self.encoder(enc_input)
        mu, logvar = enc_output[:, :self.latent_dim], enc_output[:, self.latent_dim:]
        z = self.reparameterize(mu, logvar)
        # Decode
        dec_input = torch.cat([z, condition], dim=1)
        reconstructed = self.decoder(dec_input)
        return reconstructed, mu, logvar

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cvae = CVAE().to(device)
print("CVAE model initialized.")

In [28]:
from transformers import GPT2Tokenizer

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set the padding token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

# Now tokenize the input text
text = "Hello, world!"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Check the tokenized output
print(inputs)

{'input_ids': tensor([[15496,    11,   995,     0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [29]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Define the get_embeddings function
def get_embeddings(text):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    # Check if tokenization produced any tokens
    if inputs['input_ids'].shape[1] == 0:
        return torch.zeros(1, 768)  # Return a zero vector if no tokens
    
    # Get embeddings from the model
    with torch.no_grad():
        outputs = model(**inputs)
        emb = outputs.last_hidden_state.mean(dim=1)  # Average over sequence length
    return emb

# Test the function
text = "Hello, world!"
emb = get_embeddings(text)
print(emb.shape)  # Should output: torch.Size([1, 768])

torch.Size([1, 768])


In [30]:
# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Ensure padding token is defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Padding token set to eos_token.")
else:
    print("Padding token is already set.")

Padding token set to eos_token.


In [2]:
from datasets import load_dataset
import pandas as pd
import re

# Load dataset
ds = load_dataset("MohammadOthman/mo-customer-support-tweets-945k")
data = ds['train'].select(range(100000)).to_pandas()

# Preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

data['input'] = data['input'].apply(preprocess_text)
data['output'] = data['output'].apply(preprocess_text)

# Use a subset for faster testing
data_subset = data.head(1000)

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import torch

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define dataset class
class CustomerServiceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = "respond to query: " + self.data.iloc[idx]['input']
        target_text = self.data.iloc[idx]['output']
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors="pt", max_length=512, truncation=True)
        return {"input_ids": input_ids.squeeze(), "labels": target_ids.squeeze()}

# Create dataset and dataloader
dataset = CustomerServiceDataset(data_subset)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Set up device and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Fine-tune the model
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


RuntimeError: stack expects each tensor to be equal size, but got [13] at entry 0 and [35] at entry 1

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import torch

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define dataset class
class CustomerServiceDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=512, max_output_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = "respond to query: " + self.data.iloc[idx]['input']
        target_text = self.data.iloc[idx]['output']
        # Tokenize input and target texts
        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_input_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text,
            max_length=self.max_output_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        # Return input_ids and labels
        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_encoding["attention_mask"].squeeze(),
            "labels": target_encoding["input_ids"].squeeze()
        }

# Assuming 'data_subset' is a pandas DataFrame with 'input' and 'output' columns
dataset = CustomerServiceDataset(data_subset, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Set up device and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Fine-tune the model
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1, Loss: 1.1556397714614868
Epoch 2, Loss: 0.25594344675540925
Epoch 3, Loss: 0.22276294207572936


In [27]:
# Save model and tokenizer
model.save_pretrained("t5-finetuned-customer-service")
tokenizer.save_pretrained("t5-finetuned-customer-service")


('t5-finetuned-customer-service/tokenizer_config.json',
 't5-finetuned-customer-service/special_tokens_map.json',
 't5-finetuned-customer-service/spiece.model',
 't5-finetuned-customer-service/added_tokens.json')

In [28]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load model and tokenizer from saved directory
tokenizer = T5Tokenizer.from_pretrained("t5-finetuned-customer-service")
model = T5ForConditionalGeneration.from_pretrained("t5-finetuned-customer-service")

# Send model to device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [57]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [58]:
def generate_response(input_query):
    model.eval()

    input_text =  input_query

    # Tokenize input
    encoding = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )

    input_ids = encoding["input_ids"].to(model.device)
    attention_mask = encoding["attention_mask"].to(model.device)
    print(input_ids)
    print(attention_mask)
    # Generate response
    # with torch.no_grad():
    output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=50,
            num_beams=5,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id  # CRUCIAL for T5
        )
    print(output_ids)
    # Decode generated tokens
    generated_text = tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    ).strip()
    print("Output IDs:", output_ids[0])
    print("Decoded:", generated_text)

    return generated_text


In [48]:
def generate_response(input_query):
    model.eval()
    input_text = "respond to query: " + input_query
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text
def generate_response2(input_query):
    model.eval()
    input_text = "respond to query: " + input_query.strip()
    
    # Encode input
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate output
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=50,
            num_beams=5,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    # Decode output
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    return generated_text


In [59]:
print(generate_response("is the worst customer service"))

tensor([[  19,    8, 6025,  884,  313,    1]])
tensor([[1, 1, 1, 1, 1, 1]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])
Output IDs: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])
Decoded: 



NameError: name 'output_ids' is not defined

In [8]:
from transformers import GPT2Tokenizer, GPT2Model

# Initialize GPT-2 tokenizer and model
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2Model.from_pretrained("gpt2")
gpt2_model.to(device)

# Function to get embeddings
def get_gpt2_embeddings(text):
    inputs = gpt2_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = gpt2_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# # Function to calculate similarity
# def calculate_similarity(generated_text, actual_text):
#     gen_emb = get_gpt2_embeddings(generated_text)
#     act_emb = get_gpt2_embeddings(actual_text)
#     similarity = torch.nn.functional.cosine_similarity(gen_emb, act_emb, dim=1).item()
#     return similarity

from sentence_transformers import SentenceTransformer
import torch

# Load embedding model (simulate Lamina-v6 with MiniLM)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Function to calculate cosine similarity
def calculate_similarity(text1, text2):
    emb1 = embedder.encode(text1, convert_to_tensor=True)
    emb2 = embedder.encode(text2, convert_to_tensor=True)
    similarity = torch.nn.functional.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0)).item()
    return similarity

# Example usage
text_a = "I'm happy to help you reset your password."
text_b = "Sure, let me assist you with the password reset process."

score = calculate_similarity(text_a, text_b)
print("Cosine Similarity:", score)


Cosine Similarity: 0.7013758420944214


In [9]:
def categorize_response(similarity):
    if similarity > 0.8:
        return "positive"
    elif similarity > 0.5:
        return "casual"
    else:
        return "unhappy"

In [15]:
# Test on a sample
idx = 3

input_query = data_subset['input'].iloc[idx]
actual_response = data_subset['output'].iloc[idx]
generated_response = generate_response(input_query)
similarity = calculate_similarity(generated_response, actual_response)
category = categorize_response(similarity)

# Display results
print(f"Input Query: {input_query}")
print(f"Generated Response: {generated_response}")
print(f"Actual Response: {actual_response}")
print(f"Similarity Score: {similarity}")
print(f"Category: {category}")

Input Query: you gonna magically change your connectivity for me and my whole family
Generated Response: 
Actual Response: this is saddening to hear please shoot us a dm so that we can look into this for you
Similarity Score: 0.09479773044586182
Category: unhappy


In [16]:
generated_response

''